In [40]:
from Dataload import dataload
import torch.nn.functional as F

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils

from src import detect_faces, show_bboxes
from PIL import Image
import numpy as np

from torchvision import transforms

import pandas as pd
from module import densenet,transformer

from tqdm.notebook import tqdm

In [71]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")


dense_pre=densenet.densenet121(pretrained=True)

In [90]:
class Encoder(nn.Module):
    
    def __init__(self,num_frame=24,dim=1000,model=None,frame=0):
        super(Encoder, self).__init__()
        self.label=3
        if model is None:
            self.embedder=densenet.densenet121(pretrained=False)
            self.fc1=nn.Linear(1000,512)
        else:
            self.embedder=model
        if frame==1:
            self.fc1=nn.Linear(1000,256)
        else:
            self.fc1=nn.Linear(100,256)
        self.dim_frame=dim
        self.num_frame=num_frame
        self.fc1=nn.Linear(1000,512)
        self.fc2=nn.Linear(512, self.label)
        
        self.posencoding=transformer.PositionalEncoding(self.dim_frame,n_position=frame)
        self.transformer=transformer.MultiHeadAttention(d_k=128,d_v=128,n_head=2,frame=frame)
        
    def img_embedder(self,x,t,embedder):
        x1 = (embedder(x[:, t, :, :, :]))  # Pretrained_Densenet
        x1 = x1.view(x1.size(0), -1)
        
        return x1
    
    
    def check_raw(self,x,dim):
        if x.shape[-1]==dim:
            return False
        return True 
        
        return None
    def stack_frame(self,x):
        ##Transformation to: Frames*Channel*width*height

        cnn_embed_seq = []
     
        for t in range(x.size(1)):
            with torch.no_grad():
                x1=self.img_embedder(x,t,self.embedder)
                cnn_embed_seq.append(x1)            
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        
        return cnn_embed_seq
        
    def forward(self,x):
        if  self.check_raw(x,self.dim_frame):
            cnn_embed_seq=self.stack_frame(x)  
            cnn_embed_seq=self.posencoding(cnn_embed_seq) 
           # return cnn_embed_seq
        else:
            cnn_embed_seq=x
        ##Input to Transformer
        
        cnn_embed_seq=self.fc1(cnn_embed_seq) 
        ##Dimension Reduction
        print(cnn_embed_seq.shape)
        output=self.transformer(cnn_embed_seq,cnn_embed_seq,cnn_embed_seq)
        
        output,attention=output[0],output[1]
        print(output.shape)
        
        output=(output.transpose(1,2))
        output=self.fc2(F.relu(output))
        #output=F.relu(output)
        output=output.squeeze(1)
        return F.softmax(output, 1)


In [91]:
frame_model=Encoder(10,1000,dense_pre,1)

In [92]:
img_frames=torch.rand(10,3,256,256)

In [93]:
output=frame_model(img_frames.unsqueeze(0))

torch.Size([1, 10, 512])
torch.Size([1, 512, 10])


In [94]:
output.shape

torch.Size([1, 10, 3])

In [80]:
output

tensor([[[0.1009, 0.1061, 0.0938],
         [0.0983, 0.0953, 0.1001],
         [0.1021, 0.0975, 0.1032],
         [0.0993, 0.1040, 0.0973],
         [0.0994, 0.1049, 0.1034],
         [0.1043, 0.0991, 0.0989],
         [0.1016, 0.0935, 0.1003],
         [0.0961, 0.0958, 0.1044],
         [0.1024, 0.1033, 0.0978],
         [0.0956, 0.1005, 0.1008]]], grad_fn=<SoftmaxBackward>)

In [83]:
from module import video_model

model=video_model.Video_Feature(joint=False,grayscale=False,classfication=True,frame=10)


In [100]:
x=model.fc2(model.stack_frame(img_frames.unsqueeze(0)))

model.transformer(x,x,x)[0].shape

torch.Size([1, 512, 1])